In [1]:
# for i in range(4):
#     %run log_analyzer.ipynb

In [2]:
import os
import random

# Get the files randomly
# Specify the directory path
directory_path = '/media/andrew/ExternalM2/VIRAT/videos_original'

# List all files in the directory
file_list = os.listdir(directory_path)

# Randomly select 15 unique file paths
random_file_paths = random.sample(file_list, 16)

# Create the full file paths
full_file_paths = [os.path.join(directory_path, filename) for filename in random_file_paths]

print(full_file_paths)

['/media/andrew/ExternalM2/VIRAT/videos_original/VIRAT_S_010203_06_000620_000760.mp4', '/media/andrew/ExternalM2/VIRAT/videos_original/VIRAT_S_040001_02_001102_001530.mp4', '/media/andrew/ExternalM2/VIRAT/videos_original/VIRAT_S_000200_02_000479_000635.mp4', '/media/andrew/ExternalM2/VIRAT/videos_original/VIRAT_S_010111_08_000920_000954.mp4', '/media/andrew/ExternalM2/VIRAT/videos_original/VIRAT_S_010200_06_000702_000744.mp4', '/media/andrew/ExternalM2/VIRAT/videos_original/VIRAT_S_010200_05_000658_000700.mp4', '/media/andrew/ExternalM2/VIRAT/videos_original/VIRAT_S_010004_06_000547_000715.mp4', '/media/andrew/ExternalM2/VIRAT/videos_original/VIRAT_S_010202_03_000313_000355.mp4', '/media/andrew/ExternalM2/VIRAT/videos_original/VIRAT_S_050000_08_001235_001295.mp4', '/media/andrew/ExternalM2/VIRAT/videos_original/VIRAT_S_010206_00_000007_000035.mp4', '/media/andrew/ExternalM2/VIRAT/videos_original/VIRAT_S_050000_07_001014_001126.mp4', '/media/andrew/ExternalM2/VIRAT/videos_original/VIRAT

In [ ]:
# Suppress cargo build warnings
%env RUSTFLAGS=-Awarnings
for file_path in full_file_paths:
    for i in range(10):
        print("Running CRF", i)
        ! cargo run --bin evaluate_feature_detection_transcode --release --features "open-cv feature-logging" -- \
                --crf {i} --delta-t-max 76500 --frame-count-max 500 \
                --input-filename {file_path} \
                --scale 0.5 --detect-features
    # --color-input


env: RUSTFLAGS=-Awarnings
Running CRF 0
    Finished release [optimized] target(s) in 0.15s
     Running `/home/andrew/Code/adder-codec-rs/target/release/evaluate_feature_detection_transcode --crf 0 --delta-t-max 76500 --frame-count-max 500 --input-filename /media/andrew/ExternalM2/VIRAT/videos_original/VIRAT_S_010203_06_000620_000760.mp4 --scale 0.5 --detect-features`
█░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ 10/500